In [1]:
# Import des librairies
import os
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import decomposition

In [2]:
# Import des données
clicks_sample = pd.read_csv('../1-Données/clicks_sample.csv', sep=",")
clicks_folder = "../1-Données/clicks/"
articles_metadata = pd.read_csv('../1-Données/articles_metadata.csv', sep=",")
articles_embeddings = pd.read_pickle('../1-Données/articles_embeddings.pickle')

In [3]:
# Dataframe de tous les clics
df_clicks = pd.DataFrame()
for f in os.listdir(clicks_folder):
    df = pd.read_csv(clicks_folder + f)
    df_clicks = pd.concat([df_clicks, df])

In [4]:
# Entrée
id_user = 1

In [5]:
# Fonction
# Sort un dataframe comprenant les 5 articles les plus proches de l'article ainsi que leur score
def get_best_articles(id_article):
    similarities = np.squeeze(cosine_similarity(articles_embeddings[:,:], articles_embeddings[id_article,:].reshape(1, 250)))
    indices_max = np.argsort(-similarities)[1:6]
    return pd.DataFrame([indices_max, similarities[indices_max]], index=['article', 'score'])

In [6]:
# Liste des articles vus par un utilisateur
user_articles = df_clicks.loc[df_clicks['user_id']==id_user]['click_article_id']

# Création du dataframe contenant les recommandations
df_best_articles = pd.DataFrame()

# Parcours des articles vus par l'utilisateur
for i, article in user_articles.iteritems():
    # Récupère 5 plus proches articles et les concatène dans un df
    df = get_best_articles(article)
    df_best_articles = pd.concat([df_best_articles, df], axis=1)

# Tri de tous les articles de l'utilsateur selon leur score
df_best_articles.sort_values(by=['score'], axis=1, ascending=False)
df_best_articles.columns = range(len(df_best_articles.columns))

# Liste des 5 meilleures recommandations 
results = []
i = 0
j = 0
while j < 5:
    if  not df_best_articles[i]["article"] in user_articles.unique():
        results.append(int(df_best_articles[i]["article"]))
        j = j + 1
    i = i + 1

In [7]:
# Sortie
results

[60509, 235312, 235410, 108443, 234819]